# Fourtify

**Credit**: The DP0.3 data set was generated by members of the Rubin Solar System Pipelines and Commissioning teams, with help from the LSST Solar System Science Collaboration, in particular: Pedro Bernardinelli, Jake Kurlander, Joachim Moeyens, Samuel Cornwall, Ari Heinze, Steph Merritt, Lynne Jones, Siegfried Eggl, Meg Schwamb, Grigori Fedorets, and Mario Juric.



In [1]:
import numpy as np
import pandas as pd
from fourtify import Fourtify
from astropy.time import Time
from astropy import units as u
from lsst.rsp import get_tap_service

## Helper functions for DP0.3

In [2]:
def query(adql, con=None):
    """
    DB query
    """
    if con is None:
        con = get_tap_service("ssotap")
    return con.search(adql).to_table().to_pandas()

In [3]:
def create_observations(time_ranges,dist_ranges):
    """
    Query DP0.3 data for observations in a time range and distance range.
    """

    q = """
        SELECT ds.ra,ds.dec,ds.midPointMjdTai,ds.diaSourceId,ds.ssObjectId,
        sss.heliocentricDist,
        sss.heliocentricX-sss.topocentricX as observerX,
        sss.heliocentricY-sss.topocentricY as observerY,
        sss.heliocentricZ-sss.topocentricZ as observerZ
        FROM dp03_catalogs_1yr.DiaSource AS ds
        INNER JOIN dp03_catalogs_1yr.SSSource AS sss
        ON ds.diaSourceId = sss.diaSourceId
        WHERE ds.midPointMjdTai >= {:f} AND ds.midPointMjdTai <= {:f} 
        AND sss.heliocentricDist > {:f} AND sss.heliocentricDist <= {:f}
        ORDER BY midPointMjdTai,ds.diaSourceId ASC
    """
    
    obs = query(q.format(time_ranges[0],time_ranges[1],dist_ranges[0],dist_ranges[1]))
    obs[['observerX','observerY','observerZ']] = obs[['observerX','observerY','observerZ']].astype(float)
                
    return obs

## Get sources for Fourtify to search

In [4]:
#get all DP0.3 transient sources in a 45 day window from database 
#NOTE: this takes like 10 minutes because we're loading ~5.7MM sources!

time_interval = [60218.00491,60218.00491+45] #mjd interval to get diaSources on
range_interval = [0,1e10] #all distances (AU)
obs = create_observations(time_interval,range_interval)

In [5]:
#convert obs for Fourtify init
#location lon,lat of observatory Gemini (I11) right next to Rubin

obs_radecs = np.array(obs[['ra','dec']]) #degrees
obs_times = Time(obs['midPointMjdTai'],format='mjd',scale='utc',location=(289.26345*u.deg,-30.240641*u.deg)).tdb.jd #TDB jdate
obs_locs = np.array(obs[['observerX','observerY','observerZ']]) #heliocentric observer locations AU

## Calling Fourtify

In [6]:
#initialize Fourtify with transient sources to search (only have to run once)

ff = Fourtify(obs_radecs,obs_times,obs_locs)

In [7]:
#this is an example 3-nighter orbit we want to find more transient more sources for (calculated by Find_Orb)
#orbit is from OD of a 6 detection 3-nighter for ssObjectId:-7978867435213711595 with a ~5 day arc

#elems:('a','e','i','node','peri','M') units:(AU,float,deg,deg,deg,deg)
elems = [2.2156451386002, 0.166428121775, 4.8293490744869, 93.2443107848838, 304.6233793224856, 348.7030408602304]
epoch = 2460238.5 #TDB jdate epoch of above orbit

#these are the observations the orbit above was comupted from
obs.loc[[3275344, 3323484, 3616951, 3684100, 3972996, 4017229]]

,ra,dec,midPointMjdTai,diaSourceId,ssObjectId,heliocentricDist,observerX,observerY,observerZ
3275344,19.712111,-2.379780,60233.11936,5779554338486750538,-7978867435213711595,1.860026,0.923483,0.345116,0.149577
3323484,19.706260,-2.381273,60233.14308,-666928304096274332,-7978867435213711595,1.860012,0.923322,0.345467,0.149727
3616951,19.017952,-2.551895,60236.11443,-7264436804714889325,-7978867435213711595,1.858282,0.901960,0.388251,0.168277
3684100,19.012112,-2.553141,60236.13829,7358906051747058725,-7978867435213711595,1.858269,0.901778,0.388596,0.168424
3972996,18.551063,-2.650573,60238.16414,5585262173597708038,-7978867435213711595,1.857160,0.885827,0.417182,0.180814
4017229,18.545327,-2.651638,60238.18795,-4090268930407646038,-7978867435213711595,1.857147,0.885631,0.417520,0.180958


In [8]:
#search for more sources on the above orbit with Fourtify

thresh = (10,1) #1"/day deviation rate from propagated orbit up to a maximum of 10" total deviation
dradecs,fidx = ff.orbit(elems,epoch,thresh) #search for sources on an orbit

In [9]:
#the sources Fourtify found
#Fourtify extended the source count from 6 to 14, the night count from 3 to 8, and the arc from ~5 days to ~32 days!

obs.loc[fidx]

,ra,dec,midPointMjdTai,diaSourceId,ssObjectId,heliocentricDist,observerX,observerY,observerZ
2263441,20.623939,-2.110646,60229.20047,4663528428262270326,-7978867435213711595,1.862491,0.947886,0.287306,0.124508
2325433,20.618018,-2.112435,60229.22449,-5880678224015631640,-7978867435213711595,1.862475,0.947747,0.287670,0.124663
2572086,20.392969,-2.183249,60230.19580,1652531757451060663,-7978867435213711595,1.861845,0.942096,0.302128,0.130934
2634741,20.387017,-2.184967,60230.21986,-8074616670269294953,-7978867435213711595,1.861830,0.941950,0.302490,0.131088
3275344,19.712111,-2.379780,60233.11936,5779554338486750538,-7978867435213711595,1.860026,0.923483,0.345116,0.149577
3323484,19.706260,-2.381273,60233.14308,-666928304096274332,-7978867435213711595,1.860012,0.923322,0.345467,0.149727
3616951,19.017952,-2.551895,60236.11443,-7264436804714889325,-7978867435213711595,1.858282,0.901960,0.388251,0.168277
3684100,19.012112,-2.553141,60236.13829,7358906051747058725,-7978867435213711595,1.858269,0.901778,0.388596,0.168424
3972996,18.551063,-2.650573,60238.16414,5585262173597708038,-7978867435213711595,1.857160,0.885827,0.417182,0.180814
4017229,18.545327,-2.651638,60238.18795,-4090268930407646038,-7978867435213711595,1.857147,0.885631,0.417520,0.180958
